In [1]:
import numpy as np
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline, FeatureUnion, Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
from collections import namedtuple
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE, SelectFromModel
from sklearn.metrics import fbeta_score
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import f1_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
import seaborn as sns

In [2]:
Grid = namedtuple("Grid", ['model', 'param_grid'])

# Notes
- Once i recieve the data - offset the target column by 1 so one year predicts the next
- Drop all polygon data & assume we have sufficient features to predict the next year
- Each row represents the status of a given polygon & the incidents that year

In [3]:
def get_next_year(year, region, df):
    if year+1 not in set(df[YEAR_COL]):
        return float('nan')
    return df.loc[(df[YEAR_COL]==year+1) & (df[REGION_COL]==region), TARGET].values[0]

In [4]:
df = pd.read_csv(r"C:\Users\olive\Documents\GitHub\MontrealFireSafetyProject\data\all_data_clean.csv", index_col=0)

TARGET = 'Building_fire'
YEAR_COL = 'Year'
REGION_COL = 'Poly_Key'

df[REGION_COL] = df[REGION_COL].astype('str')
df['Next Year'] = df['Year'].apply(lambda x: int(x+1))

left = df.copy()
right = df.copy()

left['idx'] = left.apply(lambda row: "{}_{}".format(row['Poly_Key'], row['Year']), axis=1)
right['idx'] = right.apply(lambda row: "{}_{}".format(row['Poly_Key'], row['Next Year']), axis=1)

left = left.set_index(['idx'], drop=True)
right = right.set_index(['idx'], drop=True)

tot = left.join(right, lsuffix='_nextyear', rsuffix='_thisyear', how='inner')
tot = tot[['Poly_Key_nextyear', 'Building_fire_nextyear', 'incident_nextyear', 
           *[x for x in tot.columns if x[-9:]=='_thisyear']]]

tot = tot.drop(['Next Year_thisyear'], axis=1)
tot = tot.set_index(['Poly_Key_thisyear', 'Year_thisyear'], drop=True)

df = tot.reset_index(drop=True).copy()
X, y = df.drop([TARGET + '_nextyear'], axis=1), df[TARGET + '_nextyear']
#y = y.apply(lambda x: 1 if x > 0 else 0)

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [6]:
X_train.head()

,Poly_Key_nextyear,incident_nextyear,CFSAUID_thisyear,Building_fire_thisyear,incident_thisyear,Desc_gr_clean_Alarmes-incendies_thisyear,Desc_gr_clean_Autres incendies_thisyear,Desc_gr_clean_Fausses alertes/annulations_thisyear,Desc_gr_clean_Nouveau_thisyear,Desc_gr_clean_Premier répondant_thisyear,...,gr_0_to_14_years_thisyear,gr_15_to_29_ years_thisyear,gr_30_to_64_years_thisyear,gr_65_years_+_thisyear,gr_married_commomlawpartner_thisyear,income_per_capita_thisyear,Income_0_to_35K_thisyear,Income_35K_to_70K_thisyear,Income_70K+_thisyear,population tax payers_thisyear
272491,POLYGON ((-73.64259836850309 45.51656488644628...,3.0,H3P,0.0,2.0,0,0,0,0,2.0,...,0.19,0.17,0.44,0.20,0.48,103396.538462,0.41,0.20,0.39,7800
141034,POLYGON ((-73.55490805567743 45.51530932546331...,2.0,H2L,0.0,3.0,2,1,0,0,0.0,...,0.07,0.24,0.56,0.09,0.28,44515.005870,0.56,0.26,0.18,19760
170330,"POLYGON ((-73.55831637045785 45.5063732320102,...",0.0,H2Z,0.0,0.0,0,0,0,0,0.0,...,0.05,0.23,0.48,0.23,0.46,56547.087379,0.59,0.18,0.23,2060
168020,POLYGON ((-73.58546045467119 45.62559625146157...,4.0,H1E,0.0,2.0,0,0,0,0,2.0,...,0.16,0.19,0.46,0.19,0.44,36313.245561,0.60,0.29,0.11,35030
246016,POLYGON ((-73.63801353808631 45.46082930327982...,3.0,H4B,0.0,6.0,1,0,0,0,3.0,...,0.16,0.19,0.50,0.15,0.39,38694.444444,0.64,0.21,0.15,15840


In [7]:
X_train.columns

Index(['Poly_Key_nextyear', 'incident_nextyear', 'CFSAUID_thisyear',
       'Building_fire_thisyear', 'incident_thisyear',
       'Desc_gr_clean_Alarmes-incendies_thisyear',
       'Desc_gr_clean_Autres incendies_thisyear',
       'Desc_gr_clean_Fausses alertes/annulations_thisyear',
       'Desc_gr_clean_Nouveau_thisyear',
       'Desc_gr_clean_Premier répondant_thisyear',
       'Desc_gr_clean_Sans incendie_thisyear', 'Num_units_total_thisyear',
       'Num_units_mean_thisyear', 'Num_units_median_thisyear',
       'population_thisyear', 'total_dwellings_thisyear',
       'gr_0_to_14_years_thisyear', 'gr_15_to_29_ years_thisyear',
       'gr_30_to_64_years_thisyear', 'gr_65_years_+_thisyear',
       'gr_married_commomlawpartner_thisyear', 'income_per_capita_thisyear',
       'Income_0_to_35K_thisyear', 'Income_35K_to_70K_thisyear',
       'Income_70K+_thisyear', 'population tax payers_thisyear'],
      dtype='object')

In [8]:
numerical_features = [
       #'Building_fire_thisyear', 'incident_thisyear',
       'Desc_gr_clean_Alarmes-incendies_thisyear',
       'Desc_gr_clean_Autres incendies_thisyear',
       'Desc_gr_clean_Fausses alertes/annulations_thisyear',
       'Desc_gr_clean_Nouveau_thisyear',
       'Desc_gr_clean_Premier répondant_thisyear',
       'Desc_gr_clean_Sans incendie_thisyear', 'Num_units_total_thisyear',
       'Num_units_mean_thisyear', 'Num_units_median_thisyear',
       'population_thisyear', 'total_dwellings_thisyear',
       'gr_0_to_14_years_thisyear', 'gr_15_to_29_ years_thisyear',
       'gr_30_to_64_years_thisyear', 'gr_65_years_+_thisyear',
       'gr_married_commomlawpartner_thisyear', 'income_per_capita_thisyear',
       'Income_0_to_35K_thisyear', 'Income_35K_to_70K_thisyear',
       'Income_70K+_thisyear', 'population tax payers_thisyear'
]

categorical_features = []

In [9]:
categorical_pipeline = Pipeline(steps=[('one_hot_encoder', OneHotEncoder(sparse=False, 
                                                                         drop='first',
                                                                         handle_unknown='error'))])

In [10]:
numerical_pipeline = Pipeline(steps=[('min_max_scalar', MinMaxScaler())])

In [11]:
prep_pipeline = ColumnTransformer([
    ("cat", categorical_pipeline, categorical_features),
    ("num", numerical_pipeline, numerical_features)
])

In [12]:
full_pipeline = Pipeline(steps=[('prep', prep_pipeline),
                                #('feature_engineering', SelectFromModel(estimator=DecisionTreeClassifier(), threshold='0.5*mean')),
                                ('estimator', Ridge())])

In [13]:
full_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('prep',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('cat',
                                                  Pipeline(memory=None,
                                                           steps=[('one_hot_encoder',
                                                                   OneHotEncoder(categorical_features=None,
                                                                                 categories=None,
                                                                                 drop='first',
                                                                                 dtype=<class 'numpy.float64'>,
                                                                                 handle_unknown='error',
                                    

In [14]:
y_pred = full_pipeline.predict(X_test)

In [15]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error

In [16]:
mean_absolute_error(y_test, y_pred)

0.09562164691709273

### Baseline: Predict next year there will be an incident if there was this year

In [17]:
y_b1 = X_test[TARGET+'_thisyear']

In [18]:
mean_absolute_error(y_test, y_b1)

0.09340076523847407

In [19]:
feature_importances = dict(zip(numerical_features, full_pipeline.steps[-1][1].coef_))

In [20]:
feature_importances

{'Desc_gr_clean_Alarmes-incendies_thisyear': 0.7635888559075151,
 'Desc_gr_clean_Autres incendies_thisyear': 0.21000017359973544,
 'Desc_gr_clean_Fausses alertes/annulations_thisyear': 0.055082842765944565,
 'Desc_gr_clean_Nouveau_thisyear': -0.043329119485634573,
 'Desc_gr_clean_Premier répondant_thisyear': 0.15798648507618246,
 'Desc_gr_clean_Sans incendie_thisyear': 1.2554087364564577,
 'Num_units_total_thisyear': 1.0385081550821866,
 'Num_units_mean_thisyear': 0.3115466667561198,
 'Num_units_median_thisyear': -0.1827054277325416,
 'population_thisyear': 0.026954381358218506,
 'total_dwellings_thisyear': 0.009936579732491539,
 'gr_0_to_14_years_thisyear': -0.0054715280751750596,
 'gr_15_to_29_ years_thisyear': -0.09807933875546634,
 'gr_30_to_64_years_thisyear': 0.06229921946388537,
 'gr_65_years_+_thisyear': -0.017217999046224005,
 'gr_married_commomlawpartner_thisyear': -0.07555548496703798,
 'income_per_capita_thisyear': 0.012251090174588227,
 'Income_0_to_35K_thisyear': 0.135218